In [ ]:
# sampling
from sklearn import datasets
import numpy as np

# example
# iris
iris = datasets.load_iris()
test=iris.data
inc_inside = iris

In [ ]:
def rearrange(arg_1, arg_2):
    if isinstance(arg_1, (np.ndarray)) == False:
        raise ValueError("input must be array")
    if isinstance(arg_2, (np.ndarray)) == False:
        raise ValueError("input must be array")
    order = np.argsort(arg_1, axis = None)
    arg_2_rearranged = [arg_2[i] for i in order]
    return np.array(arg_2_rearranged)


def sample(data, data_instance, col= None, sampling = None):
    #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    if sampling == True:
        _to_s_ = data[:,col].copy()
        data = rearrange(_to_s_ , data)
    #@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    
    # choose data and target and rerrange
    def _choose_index_(_data):
        # set seed
        rng = np.random.default_rng(seed = 1)
        _index = rng.choice(len(_data)-1, size= (round(len(_data)*0.7),) , replace=False)
        return _index

    r_d = []
    r_t = []
    for i in set(data_instance.target):
        d_1 = data[np.where(data_instance.target==i)]
        t_1 = data_instance.target[np.where(data_instance.target==i)]
        c_i = _choose_index_(d_1)
        r_d.append(d_1[c_i])
        r_t.append(t_1[c_i])
    r_d_c = r_d[0]
    r_t_c = r_t[0]
    for ind in range(0, len(r_d)-1):
        # rerranged data
        r_d_c = np.vstack((r_d_c,r_d[ind+1]))
        # rerranged target
        r_t_c = np.append(r_t_c,r_t[ind+1])
    #--------------------------------
    return r_d_c


h, w = test.shape
samples__ = []
data_s__ = sample(test, inc_inside, col=None, sampling = False)
samples__.append(data_s__)
for i in range(0,w):
    data_s_ = sample(test, inc_inside, col=i , sampling = True)
    samples__.append(data_s_)


from scipy.stats import ks_2samp
# Kolmogorov-Smirnov test 
pvalue_st = []
reuslt_1 = []
for sample_i in samples__:
    for col_ in range(0,w):
        # Apply Kolmogorov-Smirnov test to every column of original dataset and corresponding column of sample
        reuslt_1.append(ks_2samp(test[:,col_], sample_i[:,col_]).pvalue)  
    reuslt_1 = np.array(reuslt_1)
    pvalue_st.append(np.mean(reuslt_1))
    # reset
    reuslt_1 = []
pvalue_st = np.array(pvalue_st)

In [ ]:
pvalue_st

In [ ]:
# For original dataset simply do 2. For every column of original dataset firstly do 1 and then 2. Then there will be n+1 samples 
result___ = []
result___.append(sample(test, inc_inside, col=None, sampling = False))
h, w = test.shape
for i in range(0, w):
    data_s_0 = sample(test, inc_inside, col=i, sampling = True)
    result___.append(data_s_0)

In [ ]:
# Euclidean distance for mean values between all columns of original dataset and n+1 samples 
a_ = []
for data in result___:
    a_.append(np.linalg.norm(np.mean(test, axis = 0) - np.mean(data, axis = 0)))

In [ ]:
# Euclidean distance for variances values between all columns of original dataset and n+1 samples
b_ = []
for data in result___:
    b_.append(np.linalg.norm(np.var(test, axis = 0) - np.var(data, axis = 0)))

In [ ]:
a_ = np.array(a_)
b_ = np.array(b_)

In [ ]:
# index of sorted column for those samples with pvalue>0.997
# -1 means simply do 2
np.array(np.where(pvalue_st>0.997))-1

In [ ]:
# Choose those samples with pvalue>0.997 and select a_*b_ of chosen samples.
(a_*b_)[np.where(pvalue_st>0.997)] 

In [ ]:
# verify sample by graphical presentation, pay attention to tail distribution and extreme values
# for too obvious difference between sample and orginal datasaet on tail distribution and extreme values, sample needs to be reselected. 
result___ = []
data_s = test
data_s_0 = sample(test, inc_inside, col=None, sampling = False)
result___.append(data_s_0)

get_ipython().run_line_magic('matplotlib', 'notebook')
import matplotlib
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10,10),dpi=200)
ax = fig.add_subplot(5,3,1)
ax.boxplot(data_s)
ax_0 = fig.add_subplot(5,3,2)
ax_0.boxplot(data_s_0)


ax.tick_params(labelsize = 'xx-small')
ax.set_ylim(bottom=0.3, top=20)
ax.set_xlabel('original dataset', size = 'xx-small')
ax_0.tick_params(labelsize = 'xx-small')
ax_0.set_ylim(bottom=0.3, top=20)
ax_0.set_xlabel('sample', size ='xx-small' )

plt.show()

In [ ]:
# record index of column to be sorted: -1 (-1 means simply do step 2)
# seed = 1